# Convective Mode Occurences - Deltas - Linear Vs. Cellular Ratios

In [1]:
import pandas as pd
import xarray as xr
import glob
import numpy as np
from copy import deepcopy

## Load the Data

In [2]:
#iterate through the epochs
epochs = ['HIST', 'MID4p5', 'MID8p5', 'END4p5', 'END8p5']

for epoch in epochs:
    
    #load wrf-bcc dataframes
    locals()[f'df_{epoch}'] = pd.concat(map(pd.read_csv, glob.glob(f'/home/scratch/jcorner1/Thesis/future_modes/modes/{epoch}/*')), ignore_index=True)
    locals()[f'df_{epoch}']['Time'] = pd.to_datetime(locals()[f'df_{epoch}']['Time'])
    locals()[f'df_{epoch}']['Month'] = pd.to_datetime( locals()[f'df_{epoch}']['Time']).dt.month

### Convert to Convective Days

In [ ]:
seasons = ['DJF', 'MAM', 'JJA', 'SON']
modes = ['Cellular', 'Multicell', 'Linear', 'Other']

#iterate through each epoch
for epoch in epochs:
    for mode in modes:
        geog = xr.open_dataset("/home/scratch/WRF_BCC/geography/geo_em.d01.nc")
        df_wrf = locals()[f'df_{epoch}'][(locals()[f'df_{epoch}']['Mode'] == mode)]

        for season in seasons:
            print(epoch,mode, season)
            if season == 'DJF':
                df_seas = df_wrf[(df_wrf['Month'].isin([12,1,2]))]
            elif season == 'MAM':
                df_seas = df_wrf[(df_wrf['Month'].isin([3,4,5]))]
            elif season == 'JJA':
                df_seas = df_wrf[(df_wrf['Month'].isin([6,7,8]))]
            elif season == 'SON':
                df_seas = df_wrf[(df_wrf['Month'].isin([9,10,11]))]
            uhs = []

            #iterate through the dataframes
            for did, day in df_seas.resample('24h', origin='1990-10-01 12:00:00', on='Time'):

                results = np.zeros(shape=(899, 1399))
                results[day.y.values.astype(int), day.x.values.astype(int)] = 1
                uh_uvv = deepcopy(geog)
                uh_uvv = uh_uvv.assign_coords({'Time': np.array([did])})
                uh_uvv = uh_uvv.assign(UVV_DAYS = (('Time', 'south_north', 'west_east'), np.expand_dims(results, axis=0)))
                uh_uvv = uh_uvv[['CLAT', 'CLONG', 'UVV_DAYS']].coarsen(south_north=20, west_east=20, boundary='trim').mean()
                uh_uvv['UVV_DAYS'] = 1*(uh_uvv['UVV_DAYS'] > 0)

                uhs.append(uh_uvv)

            locals()[f'uhs_{epoch}_{season}_{mode}'] = xr.concat(uhs, dim='Time')

            if epoch != 'HIST': 
                locals()[f'delta_{epoch}_{season}_{mode}'] = (np.sum(locals()[f'uhs_{epoch}_{season}_{mode}'].UVV_DAYS.values, axis=0) - np.sum(locals()[f'uhs_HIST_{season}_{mode}'].UVV_DAYS.values, axis=0)) / 15

HIST Cellular DJF
HIST Cellular MAM


## Convective Mode Occurences

## Convective Mode Deltas

## Cellular Vs. Linear Ratios